# No Claim Discount Scheme

## 1. Assumptions

In [56]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.3f}'.format

In [57]:
# The code was removed by Watson Studio for sharing.

In [58]:
body = client_45e1452aa04c45e4a6c774eae7e79e50.get_object(Bucket='application-donotdelete-pr-yxymcavya8mx0j',Key='band.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
band = pd.read_csv(body, sep=';')
band

,band,freq,sales
0,Very High,0.400,0.080
1,High,0.270,0.180
2,Medium,0.200,0.340
3,Low,0.150,0.260
4,Very Low,0.050,0.140


In [59]:
body = client_45e1452aa04c45e4a6c774eae7e79e50.get_object(Bucket='application-donotdelete-pr-yxymcavya8mx0j',Key='month-sales.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
sales = pd.read_csv(body, sep=';')
sales

,month,sales
0,January,0.120
1,February,0.100
2,March,0.080
3,April,0.080
4,May,0.060
5,June,0.050
6,July,0.050
7,August,0.060
8,September,0.080
9,October,0.120


In [60]:
body = client_45e1452aa04c45e4a6c774eae7e79e50.get_object(Bucket='application-donotdelete-pr-yxymcavya8mx0j',Key='ncd-loss.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
loss = pd.read_csv(body, sep=';')
loss

,ncd,loss
0,0,0.350
1,1,0.230
2,2,0.180
3,3,0.150
4,4,0.120
5,5,0.100
6,6,0.090
7,7,0.070
8,8,0.050
9,9,0.030


In [61]:
body = client_45e1452aa04c45e4a6c774eae7e79e50.get_object(Bucket='application-donotdelete-pr-yxymcavya8mx0j',Key='month-claims.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
claims = pd.read_csv(body, sep=';')
claims

,month,claims
0,January,0.150
1,February,0.100
2,March,0.070
3,April,0.070
4,May,0.050
5,June,0.030
6,July,0.030
7,August,0.030
8,September,0.050
9,October,0.120


In [62]:
yearly_sales=1000000

## 2. Functions

### Customer database in December 2020

In [63]:
# 'year' and 'month' correspond to when customers bought the policy
# 'band' corresponds to claim frequency (1 for Very High to 5 for Very Low)
# '0' to '10' show customer headcount per NCD level

customers_2020=pd.DataFrame()
customers_2020=pd.DataFrame(columns=['year', 'month', 'band', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
for j in range (9, 13):
    for i in range (1, 6):
        index='2020'+str(j)+str(i)
        customers_2020.loc[index]=[2020, j, i, yearly_sales*band.iloc[i-1,2]*sales.iloc[j-1,1],0,0,0,0,0,0,0,0,0,0]
customers_2020.head()

,year,month,band,0,1,2,3,4,5,6,7,8,9,10
202091,"2,020.000",9.000,1.000,"6,400.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
202092,"2,020.000",9.000,2.000,"14,400.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
202093,"2,020.000",9.000,3.000,"27,200.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
202094,"2,020.000",9.000,4.000,"20,800.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
202095,"2,020.000",9.000,5.000,"11,200.000",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


### NCD transition matrix (yearly)

In [64]:
# transition matrix based on Markov chains using NCD scale rules, Poisson distribution and loss probability

def matrix(band_number):
    # retrieves claim frequency based on band number
    freq=band.iloc[band_number-1,1]
    # builds transition matrix
    matrix=np.mat(np.zeros((11,11)))
    matrix[0,0]=(1-np.exp(-freq))
    for j in range(1,4):
        matrix[j,0]=matrix[j-1,0]-np.exp(-freq)*freq**(j)/np.math.factorial(j)
    for j in range(0,10):    
        matrix[0+j,1+j]=np.exp(-freq)*(freq**0)/np.math.factorial(0)
    for j in range(0,9):    
        matrix[1+j,1+j]=np.exp(-freq)*(freq**1)/np.math.factorial(1)
    for j in range(0,9):    
        matrix[2+j,1+j]=np.exp(-freq)*(freq**2)/np.math.factorial(2)
    for j in range(0,8):    
        matrix[3+j,1+j]=np.exp(-freq)*(freq**3)/np.math.factorial(3)
    for j in range(0,7):    
        matrix[4+j,1+j]=1-np.exp(-freq)*(1+freq+(freq**2)/np.math.factorial(2)+(freq**3)/np.math.factorial(3))  
    matrix[10,10]=np.exp(-freq)*(1+freq)
    # applies loss probability based on new NCD level
    for i in range(0,11):
        for j in range(0,11):
            matrix[i,j]=matrix[i,j]*(1-loss.iloc[j,1])
    return(matrix)

### Customer database in December of given year

In [65]:
def customers(year):
    customers=customers_2020.copy()
    # creates new rows for sales after December 2020
    for y in range(2021,year+1):
        for j in range (1, 13):
            for i in range (1, 6):
                index=str(y)+str(j)+str(i)
                customers.loc[index]=[y, j, i, yearly_sales*band.iloc[i-1,2]*sales.iloc[j-1,1],0,0,0,0,0,0,0,0,0,0]
    # projects customers to end of current year
    for i in customers.index:
        list_old=customers.loc[i][3:]
        vect_old=list_old.to_numpy()
        vect_new=np.dot(vect_old,np.linalg.matrix_power(matrix(int(i[-1])),year-int(i[0:4])))
        list_new=vect_new.tolist()
        customers.loc[i][3:]=list_new[0]
    return(customers)

### NCD distribution in December of given year

In [66]:
def ncd(year):
    split=pd.DataFrame(np.zeros((12,2)),index=[0,1,2,3,4,5,6,7,8,9,10,'Total'],columns=['ncd_count', 'ncd_proportion'])
    x=customers(year).copy()
    # summarizes headcount based on NCD level
    for i in range(0,11):
        for j in x.index:
            split.iloc[i,0]=split.iloc[i,0]+x.loc[j][int(i)]
        split.iloc[11,0]=split.iloc[11,0]+split.iloc[i,0]
    # calculates proportion for each NCD level
    for i in range(0,11):
        split.iloc[i,1]=split.iloc[i,0]/split.iloc[11,0]
    split.iloc[11,1]=1
    return(split)

### Claim frequency in December of given year

In [67]:
def freq(year):
    freq=pd.DataFrame(np.zeros((5,2)), index=['Very High','High','Medium','Low','Very Low'],columns=['end_year','new_sales'])
    total=0
    y=customers(year).copy()
    x=y.groupby(['band']).sum()
    # determines distribution of claim frequency at end of year
    for i in range(0,5):
        freq.iloc[i,0]=x[range(0,11)].sum(axis=1).iloc[i]
        total=total+freq.iloc[i,0]
    # shows claim frequency distribution for new sales
    for i in range(0,5):
        freq.iloc[i,0]=freq.iloc[i,0]/total
        freq.iloc[i,1]=band.iloc[i,2]
    return(freq)

## 3. Exercise

### Question a

In [68]:
ncd(2021)

,ncd_count,ncd_proportion
0,"1,045,146.774",0.804
1,"254,518.437",0.196
2,0.000,0.000
3,0.000,0.000
4,0.000,0.000
5,0.000,0.000
6,0.000,0.000
7,0.000,0.000
8,0.000,0.000
9,0.000,0.000


### Question b

In [69]:
ncd(2022)

,ncd_count,ncd_proportion
0,"1,121,801.656",0.564
1,"693,278.579",0.349
2,"173,771.330",0.087
3,0.000,0.000
4,0.000,0.000
5,0.000,0.000
6,0.000,0.000
7,0.000,0.000
8,0.000,0.000
9,0.000,0.000


### Question c

A subtle shift from higher to lower frequencies can be observed.

In [70]:
freq(2029)

,end_year,new_sales
Very High,0.074,0.080
High,0.175,0.180
Medium,0.339,0.340
Low,0.265,0.260
Very Low,0.148,0.140


### Question d

In [71]:
# number of 2020 customers left at the end of 2029
y=customers(2029).copy()
x=y[y['year']==2020].groupby(['year']).sum()
first_2029=x[range(0,11)].sum(axis=1).iloc[0]

# initial number of 2020 customers
x=customers_2020[customers_2020['year']==2020].groupby(['year']).sum()
first_2020=x[range(0,11)].sum(axis=1).iloc[0]

print('December 2020 = {:,.0f} \nDecember 2029 = {:,.0f} \nRatio = {:,.4f}'.format(first_2020, first_2029, first_2029/first_2020))

December 2020 = 400,000 
December 2029 = 106,746 
Ratio = 0.2669


### Question e

As shown below, the NCD headcount and distribution do not seem to stabilise over time.

In [72]:
ncd(2050)

,ncd_count,ncd_proportion
0,"1,143,906.488",0.127
1,"833,888.066",0.093
2,"654,948.360",0.073
3,"537,176.720",0.060
4,"459,349.615",0.051
5,"403,611.503",0.045
6,"359,638.945",0.040
7,"329,660.870",0.037
8,"315,101.364",0.035
9,"355,401.053",0.039


In [73]:
ncd(2100)

,ncd_count,ncd_proportion
0,"1,143,906.490",0.081
1,"833,888.081",0.059
2,"654,948.464",0.046
3,"537,177.383",0.038
4,"459,353.874",0.032
5,"403,638.791",0.029
6,"359,816.038",0.025
7,"331,154.068",0.023
8,"327,150.639",0.023
9,"459,294.805",0.032
